Import necessary packages and open up freshly cleaned dataset

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Search
import ipyleaflet
import matplotlib.pyplot as plt
from matplotlib import colormaps
import matplotlib.colors as mcolors
import json

In [ ]:
wind_turbines = gpd.read_file('../DataPrep/wind_turbines.geojson')

In [ ]:
# Define functions for later use

# modify pop-up settings to only show relevant information, i.e. if N/A, dont show
# but only for columns related to planning process i.e. from planning application withdrawn column to operational column (last column)
# Function to create conditional popups, excluding N/A values for columns 15+ 
#(columns covering different stages in planning process, info unneccessary for all)
def conditional_popups(gdf):
    """
    Returns a list of popup HTML strings for all rows in a GeoDataFrame.
    """
    popups = []
    for _, row in gdf.iterrows():
        popup_cond = []

        for column in row.index[:16]:
            popup_cond.append(f"<b>{column}:</b> {row[column]}")

        for column in row.index[16:]:
            value = str(row[column])
            if value != 'N/A':
                popup_cond.append(f"<b>{column}:</b> {value}")

        popups.append("<br>".join(popup_cond))
    return popups

# Function to assign marker radius based on turbine height bins
def marker_radius(height):
    try:
        height = float(height)
        if height < 50:
            return 4
        elif height < 100:
            return 7
        elif height < 150:
            return 10
        else:
            return 13
    except:
        return 2  # default if missing/invalid

# Function to assign marker colour based on a specific column
def marker_colour(gdf, column_name):
    """
    Generate a dictionary of colors for each unique category in a given column.
    """
    unique_values = gdf[column_name].dropna().unique()
    n = len(unique_values)

    # Use a colormap from matplotlib
    cmap = colormaps.get_cmap('tab20').resampled(n)
    colors = [mcolors.to_hex(cmap(i)) for i in range(n)]

    # Map each unique category to a color
    return dict(zip(unique_values, colors))

# Define a legend function based on the colours and sizes defined previously
def add_legend(map, colour_dict):
    """
    Add a custom HTML legend to the map, including both color and size information.
    """
    size_dict = {
        '0-50 meters': 4,
        '50-100 meters': 7,
        '100-150 meters': 10,
        '150+ meters': 13
    }

    # Start building the legend HTML
    legend_html = '''
        <div style="position: fixed; 
                    bottom: 50px; left: 50px; width: 250px; height: auto; 
                    border:2px solid grey; z-index:9999; font-size:14px; 
                    background-color: white; opacity: 0.7; padding: 10px;">
                    <b>Wind Turbine Legend</b><br>
                    <b>Development Status (Short)</b><br>
    '''
    
    # Add each category and its color to the legend
    for category, color in colour_dict.items():
        legend_html += f'<i style="background: {color}; width: 20px; height: 20px; display: inline-block; margin-right: 8px;"></i>{category}<br>'
    
    # Add turbine height sizing information to the legend
    legend_html += '<br><b>Turbine Height (m)</b><br>'
    for height_range, radius in size_dict.items():
        # Add circular markers in the legend that match the size of the radius
        legend_html += f'<i style="background: gray; border-radius: 50%; width: {radius * 2}px; height: {radius * 2}px; display: inline-block; margin-right: 8px;"></i>{height_range}<br>'
  
    legend_html += '</div>'
    
    # Add the legend to the map as a popup
    map.get_root().html.add_child(folium.Element(legend_html))

In [ ]:
# Get the list of columns with their index numbers
index_columns = [(index, column) for index, column in enumerate(wind_turbines.columns)]

# Print the list
for index, column in index_columns:
    print(f"{index}: {column}")


In [ ]:
unique_values = list(wind_turbines["Development Status (short)"].dropna().unique())
unique_values

In [ ]:
# add additional raster and vector data for viewing context

In [ ]:
#TEST
# Create the map centered around the turbines' mean coordinates
m = folium.Map(location=[
    wind_turbines.geometry.y.mean(),
    wind_turbines.geometry.x.mean()
], zoom_start=6)

# Generate color mapping based on 'Development Status (short)' column
colours = marker_colour(wind_turbines, 'Development Status (short)')

# Add CircleMarkers for wind turbines
for i, (idx, row) in enumerate(wind_turbines.iterrows()):
    popup_html = f"<b>{row['Site Name']}</b><br><b>Status:</b> {row['Development Status (short)']}"
    status = row['Development Status (short)']
    color = colours.get(status, 'black')
    radius = 2  # Default radius if not specified

    if pd.notna(row.geometry.y) and pd.notna(row.geometry.x):
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=radius,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            popup=folium.Popup(popup_html, max_width=300)
        ).add_to(m)

# Create GeoJSON layer for wind turbines (only used for search functionality)
geojson_layer = folium.GeoJson(
    wind_turbines,  # Directly use the GeoDataFrame
    style_function=lambda feature: {
        "color": "transparent",  # Make the border fully transparent
        "fillColor": "transparent",  # Make the fill fully transparent
        "fillOpacity": 0,  # Ensure it is fully transparent
        "opacity": 0  # Ensure it is fully transparent
    },
    control=False,  # Don't include this in the LayerControl
    visible=False  # Ensure it's not visible on the map
).add_to(m)

# Add Search plugin for the GeoJSON layer (for search functionality only)
search = Search(
    layer=geojson_layer,  # Pass the invisible GeoJSON layer to the search function
    geom_type="Point",
    search_label="search_all",  # Use the combined column for search
    placeholder="Search anything...",
    collapsed=False
).add_to(m)

# Add satellite basemap
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite',
    overlay=True,
    control=True  # Enable layer control for the satellite basemap
).add_to(m)

# Add Layer Control to allow users to toggle satellite layers
folium.LayerControl().add_to(m)

# Add the custom legend
add_legend(m, colours)

# Display the map
m